In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="kenneth-workspace")
exp = Experiment(workspace=ws, name="kenneth-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: kenneth-workspace
Azure region: eastus
Subscription id: 7f96438t-1br7-4xn5-892k-3155er1zt569
Resource group: Udacity


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster = "kenneth-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster)
    print('Existing Cluster found')
except Exception as e:
    print(e + "No existing cluster found. Creating a new cluster.")
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4, vm_priority='lowpriority',
                                                           idle_seconds_before_scaledown=600)
    compute_target = ComputeTarget.create(ws, cluster, config)

compute_target.wait_for_completion(show_output=True)

In [8]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
import os
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform



ps = RandomParameterSampling(parameter_space={'C': uniform(0.1, 100)})

policy = BanditPolicy(slack_factor=0.1)

est = SKLearn(source_directory='./train', entry_script='train.py', compute_target=compute_target,
                script_params={ '--C' : 1, '--max_iter': 50})


hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, 
                                     max_duration_minutes=7, est=est, policy=policy)
                            
# if "training" not in os.listdir():
#     os.mkdir("./training")

In [10]:
from azureml.core.experiment import Experiment

experiment = Experiment(ws, "kenneth-experiment")
run = experiment.submit(config=hyperdrive_config, show_output=True)

In [11]:
import joblib

best_run = run.get_best_run_by_primary_metric()
best_run.download_files(prefix='output')

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(data_url)

In [13]:
import sys
sys.path.append('train')
from train import clean_data

x, y = clean_data(data)

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)

In [15]:
import numpy as np
import pandas as pd
from azureml.core import Workspace, Dataset

training_data = np.concatenate((x_train, y_train[:, np.newaxis]), axis=-1)
training_dataframe = pd.DataFrame(data=training_data, columns=x.columns.to_list() + ['y'])
training_dataframe.to_csv('kenneth.csv', index=False)


subscription_id = '7f96438t-1br7-4xn5-892k-3155er1zt569'
resource_group = 'Udacity'
workspace_name = 'kenneth-workspace'

workspace = Workspace(subscription_id, resource_group, workspace_name)

training_data = Dataset.get_by_name(workspace, name='kenneth.csv')

In [16]:
import azureml.train.automl.utilities
from azureml.train.automl import AutoMLConfig

azureml.train.automl.utilities.get_primary_metrics('classification')

config = AutoMLConfig(
    label_column_name='y',
    task='classification',
    primary_metric='AUC_weighted',
    experiment_timeout_minutes=15,
    training_data=training_data,
    n_cross_validations=4
    )


['norm_macro_recall',
 'precision_score_weighted',
 'average_precision_score_weighted',
 'AUC_weighted',
 'accuracy']

In [17]:
experiment = Experiment(ws, "kenneth-experiment-2")
run = experiment.submit(config=config, show_output=True)

Running on local machine
Parent Run ID: AutoML_4ke630v5-67b7-49rt-826n-80j2h8va30s5

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [18]:
_ , model = run.get_output()
returned_output = joblib.dump(model, 'best_automl_model.joblib')

try:
    compute_target.delete()
except Exception as e:
    print(e)